In [3]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2023-04-06 01:19:25--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.200.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  23.9MB/s    in 7.0s    

2023-04-06 01:19:33 (20.3 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import zipfile

In [5]:
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [6]:
import os

train_horse_dir = os.path.join('./horse-or-human/horses')

train_human_dir = os.path.join('./horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_human_dir)
print(train_human_names[:10])

['horse34-7.png', 'horse49-1.png', 'horse16-6.png', 'horse20-7.png', 'horse49-5.png', 'horse40-2.png', 'horse11-6.png', 'horse23-4.png', 'horse25-4.png', 'horse49-2.png']
['human12-21.png', 'human01-19.png', 'human17-00.png', 'human16-15.png', 'human03-00.png', 'human11-24.png', 'human15-09.png', 'human14-01.png', 'human11-20.png', 'human13-20.png']


In [8]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [9]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_horse_pix = [os.path.join(train_horse_dir, fname) 
                for fname in train_horse_names[pic_index-8:pic_index]]
next_human_pix = [os.path.join(train_human_dir, fname) 
                for fname in train_human_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_horse_pix+next_human_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [10]:
## criando o modelo 
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape =(300,300,3)),
  tf.keras.layers.MaxPool2D(2,2), 

  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),

  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2), 

  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),  

  tf.keras.layers.Flatten(),

  tf.keras.layers.Dense(512, activation='relu'),

  tf.keras.layers.Dense(1, activation='sigmoid')

])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [12]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
  loss='binary_crossentropy',
  optimizer=RMSprop(learning_rate= 0.0001),
  metrics=['accuracy']
)


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
train_data_generation = ImageDataGenerator(rescale=1/255)


train_generator = train_data_generation.flow_from_directory(
       './horse-or-human/',
       target_size=(300,300),
       batch_size=128,
        class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs=15,
    verbose=1
)

Epoch 1/15
8/8 [==============================] - 22s 823ms/step - loss: 0.6800 - accuracy: 0.5362
Epoch 2/15
8/8 [==============================] - 7s 813ms/step - loss: 0.6362 - accuracy: 0.5729
Epoch 3/15
8/8 [==============================] - 7s 797ms/step - loss: 0.6205 - accuracy: 0.7397
Epoch 4/15
8/8 [==============================] - 7s 886ms/step - loss: 0.5043 - accuracy: 0.8365
Epoch 5/15
8/8 [==============================] - 8s 919ms/step - loss: 0.4874 - accuracy: 0.8154
Epoch 6/15
8/8 [==============================] - 7s 868ms/step - loss: 0.4419 - accuracy: 0.8532
Epoch 7/15
8/8 [==============================] - 6s 776ms/step - loss: 0.3656 - accuracy: 0.8910
Epoch 8/15
8/8 [==============================] - 8s 902ms/step - loss: 0.2629 - accuracy: 0.9307
Epoch 9/15
8/8 [==============================] - 8s 1s/step - loss: 0.2013 - accuracy: 0.9570
Epoch 10/15
8/8 [==============================] - 7s 1s/step - loss: 0.1917 - accuracy: 0.9410
Epoch 11/15
8/8 [=======